# Python script to do joining and manipulation of data

In [1]:
# LIBRARIES TO USE ###################################################
import pandas

In [3]:
# UTILITY FUNCTIONS ##################################################
def get_dataframe( filename ):
    return pandas.read_csv( filename )

def join_data( data_one, data_two, cols, join_type='inner', rename={} ):
    
    # RENAME THE COLUMNS FOR DATA_TWO IF NECESSARY ###################
    if rename: # EMPTY DICTIONARIES EVALUATE TO FALSE ################
        data_two = data_two.rename( index=str, columns=rename )
    
    return pandas.merge( data_one, data_two, how=join_type, on=cols )

def write( data, filename, cols=[] ):
    if len(cols) > 0:
        data.to_csv( filename, columns=cols, index=False )
    else:    
        data.to_csv( filename, index=False )

## Examples

In [4]:
# BASIC JOIN EXAMPLE #################################################
df1 = get_dataframe( 'data/NCAATourneySeeds.csv')
df2 = get_dataframe( 'data/TeamConferences.csv')
cols = ['TeamID', 'Season']
joined = join_data( df1, df2, cols=cols )
write( joined, 'cleaned/ConferenceSeeds.csv')

In [6]:
# ORGANIZE THE NCAA TOURNAMENT RESULTS BASED ON THE SEED #############
seeds = get_dataframe( 'data/NCAATourneySeeds.csv' )
results = get_dataframe( 'data/NCAATourneyCompactResults.csv' )

join_cols = ['WTeamID', 'Season']
rename = { 'TeamID':'WTeamID', 'Seed':'WSeed' }
winner_joined = join_data( results, seeds, cols=join_cols, rename=rename )

join_cols = ['LTeamID', 'Season']
rename = { 'TeamID':'LTeamID', 'Seed':'LSeed' }
both_joined = join_data( winner_joined, seeds, cols=join_cols, rename=rename )

both_joined['WSeed'] = both_joined['WSeed'].apply( (lambda x: int(x[1:3])) )
both_joined['LSeed'] = both_joined['LSeed'].apply( (lambda x: int(x[1:3])) )
both_joined['SeedDiff'] = both_joined.apply( (lambda x: x['LSeed'] - x['WSeed']), axis=1 )
both_joined['BetterSeedWon'] = both_joined.apply( (lambda x: x['WSeed'] <= x['LSeed']), axis=1 )

write_cols = ['Season', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WSeed', 'LSeed', 'SeedDiff', 'BetterSeedWon']
write( both_joined, 'cleaned/TourneyResultsWithSeeds.csv', cols=write_cols )

finished = get_dataframe( 'cleaned/TourneyResultsWithSeeds.csv' )
finished.head(5)

,Season,WTeamID,WScore,LTeamID,LScore,WSeed,LSeed,SeedDiff,BetterSeedWon
0,1985,1116,63,1234,54,9,8,-1,False
1,1985,1120,59,1345,58,11,6,-5,False
2,1985,1120,66,1242,64,11,3,-8,False
3,1985,1207,68,1250,43,1,16,15,True
4,1985,1207,63,1396,46,1,8,7,True


In [14]:
# RANKINGS DATA #######################
rankings = get_dataframe( 'data/MasseyOrdinals.csv')
rankings.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [26]:
# ORGANIZE THE NCAA TOURNAMENT RESULTS BASED ON RPI, KENPOM, AND OTHER RANKINGS ######
games = get_dataframe( 'data/NCAATourneyCompactResults.csv' )
games = games[['Season', 'WTeamID', 'LTeamID']]
games = games.loc[ games['Season'] >= 2003 ]
print(len(games.index))
for system in rankings['SystemName'].value_counts()[0:20].keys():
    system_ranking = rankings.loc[ rankings['SystemName'] == system ]
    system_ranking = system_ranking.loc[ system_ranking['RankingDayNum'] == day ]
    system_ranking = system_ranking[['Season', 'TeamID', 'OrdinalRank']]
    
    join_cols = ['WTeamID', 'Season']
    rename = { 'TeamID':'WTeamID', 'OrdinalRank':('W' + system) }
    winner_joined = join_data( games, system_ranking, cols=join_cols, rename=rename, join_type='left' )

    join_cols = ['LTeamID', 'Season']
    rename = { 'TeamID':'LTeamID', 'OrdinalRank':('L' + system) }
    both_joined = join_data( winner_joined, system_ranking, cols=join_cols, rename=rename, join_type='left' )
    games = both_joined

print( len(games.index) )
write( games, 'cleaned/TourneyResultsWithRankings.csv' )
games.head(10)

981
981


,Season,WTeamID,LTeamID,WSAG,LSAG,WMOR,LMOR,WPOM,LPOM,WDOK,...,WRPI,LRPI,WBIH,LBIH,WSEL,LSEL,WDUN,LDUN,WCNG,LCNG
0,2003,1421,1411,251,257,277,293,273,268,NaN,...,186,217,247.0,234.0,233.0,237.0,241.0,247.0,NaN,NaN
1,2003,1112,1436,2,158,4,187,3,144,NaN,...,2,144,2.0,151.0,2.0,178.0,9.0,110.0,NaN,NaN
2,2003,1113,1272,32,24,22,23,32,20,NaN,...,32,26,43.0,19.0,34.0,21.0,37.0,19.0,NaN,NaN
3,2003,1141,1166,48,18,50,33,50,27,NaN,...,44,23,38.0,17.0,48.0,22.0,32.0,27.0,NaN,NaN
4,2003,1143,1301,40,46,43,31,40,48,NaN,...,33,53,26.0,49.0,28.0,47.0,62.0,23.0,NaN,NaN
5,2003,1163,1140,23,28,17,42,23,26,NaN,...,28,19,30.0,31.0,37.0,25.0,21.0,61.0,NaN,NaN
6,2003,1181,1161,10,89,10,85,8,87,NaN,...,12,75,11.0,85.0,8.0,75.0,14.0,45.0,NaN,NaN
7,2003,1211,1153,39,42,54,39,41,35,NaN,...,43,29,42.0,52.0,40.0,43.0,72.0,67.0,NaN,NaN
8,2003,1228,1443,11,60,5,59,5,52,NaN,...,18,58,16.0,56.0,10.0,60.0,4.0,42.0,NaN,NaN
9,2003,1242,1429,4,101,2,128,4,109,NaN,...,6,79,9.0,83.0,6.0,90.0,7.0,87.0,NaN,NaN
